# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.122201e+02     1.679470e+00
 * time: 0.06345605850219727
     1     1.097129e+01     8.766797e-01
 * time: 1.3056368827819824
     2    -1.238389e+01     1.012615e+00
 * time: 1.3910250663757324
     3    -3.411631e+01     7.552049e-01
 * time: 1.4758470058441162
     4    -4.781470e+01     5.525644e-01
 * time: 1.5720560550689697
     5    -5.702902e+01     2.367172e-01
 * time: 1.6449220180511475
     6    -5.988025e+01     1.260197e-01
 * time: 1.701570987701416
     7    -6.096129e+01     6.351304e-02
 * time: 1.7700200080871582
     8    -6.137197e+01     7.379804e-02
 * time: 1.8289048671722412
     9    -6.165413e+01     3.962129e-02
 * time: 1.8856689929962158
    10    -6.185988e+01     2.616946e-02
 * time: 1.942547082901001
    11    -6.201047e+01     2.113210e-02
 * time: 2.004307985305786
    12    -6.207230e+01     1.549582e-02
 * time: 2.0603909492492676
    13    -6.213647e+01     1.280681e-02
 * time: 2.117045879364

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671094
    AtomicLocal         -18.8557740
    AtomicNonlocal      14.8522661
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485406 
    Xc                  -19.3336834

    total               -62.261666451240
